<a href="https://colab.research.google.com/github/sanntana21/TFG/blob/main/aggregated_time_series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PREDICCIONES AGREGADAS**

In [ ]:
print("v.0.0")

In [30]:
from tensorflow.keras import layers
import pandas as pd
from statistics import mean, median
import numpy as np
import pickle
import seaborn as sbn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import random as random
import plotly.express as px
import tensorflow as tf
import gzip

def set_output_precision(decimals):
  """
  format the output of the all the data structures
  with an specific number of decimals
  """
  np.set_printoptions(precision=decimals)
  into='{'+':.{}f'.format(decimals)+'}'
  pd.options.display.float_format = into.format
  pass

def plot_ts(df,dfx="Minute",dfy="METS",_title="DF minute x Mets"):
  if not isinstance(df,pd.DataFrame):
    df = pd.DataFrame({'METS': df, 'Minute': range(len(df))})

  plt.figure()
  fig = px.line(df, x = dfx, y = dfy , title = _title)
  fig.update_xaxes(
      rangeslider_visible = True,
      rangeselector = dict(
          buttons = list([
              dict(count=1,label="1y",step="year",stepmode="backward"),
              dict(count=2,label="2y",step="year",stepmode="backward"),
              dict(count=3,label="3y",step="year",stepmode="backward"),
              dict(step="all")
          ])
      )

  )
  fig.show()

def plot_predictions_vs_real(predictions, reals):
  df = pd.DataFrame()
  number_of_points = len(predictions)
  df["time"] = range(0,number_of_points)
  df["participant"] = "prediction"
  df["value"] = predictions
  for i in range(0,number_of_points):
    df.loc[number_of_points+i] = [i,"real",reals[i]]

  plt.figure(1)
  fig = px.line(df, x = "time", y = "value" , title = "predictions vs reals" , color = "participant")
  fig.update_xaxes(
        rangeslider_visible = True,
        rangeselector = dict(
            buttons = list([
                dict(count=1,label="1y",step="year",stepmode="backward"),
                dict(count=2,label="2y",step="year",stepmode="backward"),
                dict(count=3,label="3y",step="year",stepmode="backward"),
                dict(step="all")
            ])
        )

    )
  fig.show()

In [31]:
INDEXS = [{"train":[{"start":{"day":2,"hour":0,"minute":0},
                     "end":{"day":20,"hour":22,"minute":0}}
                    ],
           "validation":[{"start":{"day":22,"hour":0,"minute":0},
                     "end":{"day":24,"hour":22,"minute":0}}
                    ],
           "test":[{"start":{"day":26,"hour":0,"minute":0},
                     "end":{"day":27,"hour":22,"minute":0}}
                    ]}
          ]

In [32]:
READ_LOCAL_DATA = True
COMPUTED_OPTION = 0
SAVE_RESULTS = False
LOW_DATA = True
SPLIT_INTO_TWO_DAYS = False
MULTI_STEP_FORECAST = False
np.random.seed(42)
SPLIT = 0

In [33]:
if READ_LOCAL_DATA:
  PATH = "Resources/Individual/"
else:
    #  We start by getting access to the drive
    from google.colab import drive
    drive.mount('/content/drive')
    PATH = "/content/drive/MyDrive/TFG/Resources/Agregado/"


if LOW_DATA:
    PATH += "LowData/"


In [34]:
import gzip
documents = ['minuteY','hourY','dayY']
# with open(PATH+"minuteX"+".pkl", 'rb') as file:
#     dataX = np.array(pickle.load(file),np.float32)
file = PATH+"minuteX.pkl.gz"
dataX = np.array(pickle.load(gzip.open(file, 'rb')),np.float32)

file = PATH+documents[COMPUTED_OPTION]+".pkl.gz"
dataY = np.array(pickle.load(gzip.open(file, 'rb')),np.float32)

dataX = np.sum(dataX,axis=0,keepdims=True)
dataY = np.sum(dataY,axis=0,keepdims=True)

In [35]:
if MULTI_STEP_FORECAST:
    PREDICTED_HORIZON = 1
    if COMPUTED_OPTION == 0:
        dataX_nuevo = dataX
        JUMP = 1
    elif COMPUTED_OPTION == 1:
        JUMP = 60
        dataX_nuevo = np.ones(shape=(dataX.shape[0],24))
        for i in range(0,dataX.shape[0]):
            for j in range(0,24):
                dataX_nuevo[i,j] = np.sum(dataX[i,60*j:60*(j+1)])
    else:
        JUMP = 1440
        dataX_nuevo = np.ones(shape=(dataX.shape[0],1))
        for i in range(0,dataX.shape[0]):
            dataX_nuevo[i,:] = np.sum(dataX[i,:])

In [36]:
#We split a test set for testing
# train_test_split(dataX, dataY, test_size=TEST_SIZE)

def calculate_index(time):
  minute_index = time["day"] * 1440 + time["hour"]*60 + time["minute"]
  return minute_index

def get_split(dataX,dataY,index):
  start = calculate_index(index[0]["start"])
  end = calculate_index(index[0]["end"])
  X_split = dataX[:,start:end,:]
  y_split = dataY[:,start:end,:]
  if len(index) > 1:
    for i in range(1,index):
      start = calculate_index(index[i]["start"])
      end = calculate_index(index[i]["end"])
      X_split = np.concatenate(X_split,dataX[:,start:end,:])
      y_split = np.concatenate(y_split,dataY[:,start:end,:])
  return X_split,y_split

def train_test_validation_split(dataX,dataY,indexs):
  X_train,y_train = get_split(dataX,dataY,indexs["train"])
  X_validation,y_validation = get_split(dataX,dataY,indexs["validation"])
  X_test,y_test = get_split(dataX,dataY,indexs["test"])
  return X_train,y_train,X_validation,y_validation,X_test,y_test


X_train,y_train,X_validation,y_validation,X_test,y_test = train_test_validation_split(dataX,dataY,INDEXS[SPLIT])
print("Examples for training\n","X:",X_train.shape,"y:",y_train.shape)
print("Examples for validation\n","X:",X_validation.shape,"y:",y_validation.shape)
print("Examples for test\n","X:",X_test.shape,"y:",y_test.shape)


Examples for training
 X: (1, 27240, 1440) y: (1, 27240, 120)
Examples for validation
 X: (1, 4200, 1440) y: (1, 4200, 120)
Examples for test
 X: (1, 2760, 1440) y: (1, 2760, 120)


In [37]:
#Quitamos en este caso la primera dimension
X_train,y_train,X_validation,y_validation,X_test,y_test = [
  np.squeeze(i) for i in [X_train,y_train,X_validation,y_validation,X_test,y_test]
]

print("Examples for training\n","X:",X_train.shape,"y:",y_train.shape)
print("Examples for validation\n","X:",X_validation.shape,"y:",y_validation.shape)
print("Examples for test\n","X:",X_test.shape,"y:",y_test.shape)

Examples for training
 X: (27240, 1440) y: (27240, 120)
Examples for validation
 X: (4200, 1440) y: (4200, 120)
Examples for test
 X: (2760, 1440) y: (2760, 120)


#**DEFINICION DEL MODELO**

In [38]:
tf.random.set_seed(42)
# Setup dataset hyperparameters
HORIZON = y_test.shape[1]
WINDOW_SIZE = X_test.shape[1]

# Let's build an LSTM model with the Functional API
inputs = layers.Input(shape=(WINDOW_SIZE))
x = layers.Lambda(lambda x: tf.expand_dims(x, axis=1))(inputs) # expand input dimension to be compatible with LSTM
# print(x.shape)
# x = layers.LSTM(128, activation="relu", return_sequences=True)(x) # this layer will error if the inputs are not the right shape
x = layers.LSTM(128, activation="relu")(x) # using the tanh loss function results in a massive error
# print(x.shape)
# Add another optional dense layer (you could add more of these to see if they improve model performance)
# x = layers.Dense(32, activation="relu")(x)
output = layers.Dense(HORIZON)(x)
model_LSTM = tf.keras.Model(inputs=inputs, outputs=output, name="model_5_lstm")

model_LSTM.summary()

Model: "model_5_lstm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1440)]            0         
                                                                 
 lambda_1 (Lambda)           (None, 1, 1440)           0         
                                                                 
 lstm_1 (LSTM)               (None, 128)               803328    
                                                                 
 dense_1 (Dense)             (None, 120)               15480     
                                                                 
Total params: 818,808
Trainable params: 818,808
Non-trainable params: 0
_________________________________________________________________


In [39]:
# Compile model
model_LSTM.compile(loss="mae",
                optimizer=tf.keras.optimizers.Adam(),
             metrics=["mae"])

In [40]:
# Seems when saving the model several warnings are appearing: https://github.com/tensorflow/tensorflow/issues/47554
model_LSTM.fit(X_train,
            y_train,
            epochs=10,
            verbose=1,
            batch_size=128,
            validation_data=(X_validation, y_validation))

Epoch 1/10
213/213 [==============================] - 4s 11ms/step - loss: 5.4825 - mae: 5.4825 - val_loss: 2.6976 - val_mae: 2.6976
Epoch 2/10
213/213 [==============================] - 2s 10ms/step - loss: 2.5061 - mae: 2.5061 - val_loss: 2.7503 - val_mae: 2.7503
Epoch 3/10
213/213 [==============================] - 2s 10ms/step - loss: 2.3883 - mae: 2.3883 - val_loss: 2.5320 - val_mae: 2.5320
Epoch 4/10
213/213 [==============================] - 2s 10ms/step - loss: 2.3735 - mae: 2.3735 - val_loss: 2.5550 - val_mae: 2.5550
Epoch 5/10
213/213 [==============================] - 2s 10ms/step - loss: 2.3179 - mae: 2.3179 - val_loss: 2.3832 - val_mae: 2.3832
Epoch 6/10
213/213 [==============================] - 2s 10ms/step - loss: 2.2911 - mae: 2.2911 - val_loss: 2.3662 - val_mae: 2.3662
Epoch 7/10
213/213 [==============================] - 2s 10ms/step - loss: 2.2896 - mae: 2.2896 - val_loss: 2.3242 - val_mae: 2.3242
Epoch 8/10
213/213 [==============================] - 2s 10ms/step - 

In [41]:
def make_preds(model, input_data):
  """
  Uses model to make predictions on input_data.

  Parameters
  ----------
  model: trained model
  input_data: windowed input data (same kind of data model was trained on)

  Returns model predictions on input_data.
  """
  forecast = model.predict(input_data,verbose=2)
  return tf.squeeze(forecast) # return 1D array of predictions

predictions = make_preds(model_LSTM, X_test)

87/87 - 0s - 226ms/epoch - 3ms/step


In [42]:
print("Resultados comparando directamente las salidas")
print(y_test.shape)
print(predictions.shape)
print('\033[1m' + "MSE: " + str(mean_squared_error(y_test,predictions)) + '\033[0m')
print('\033[1m' + "MAE: " + str(mean_absolute_error(y_test,predictions)) + '\033[0m')
print(np.mean(y_test))

Resultados comparando directamente las salidas
(2760, 120)
(2760, 120)
MSE: 10.375926
MAE: 2.3592067
38.235935


In [43]:
SPLIT_INTO_TWO_DAYS = True
if not MULTI_STEP_FORECAST:
  print("Resultados poblacionales cada dos horas")
  print('\033[1m' + "MSE: " + str(mean_squared_error(np.sum(y_test,axis=1),np.sum(predictions,axis=1))) + '\033[0m')
  print('\033[1m' + "MAE: " + str(mean_absolute_error(np.sum(y_test,axis=1),np.sum(predictions,axis=1))) + '\033[0m')
  print("Ejemplos aleatorios")
  list_of_MAE = [  mean_absolute_error(predictions[i],y_test[i]) for i in range(0,len(y_test)) ]
  list_of_values = sorted(list_of_MAE)
  mean_value = mean(list_of_MAE)
  closest_value = min(list_of_MAE, key=lambda x: abs(x - mean_value))
  # Crear un array de índices
  indices = [list_of_MAE.index(list_of_values[-1]),
             list_of_MAE.index(closest_value) ,
            list_of_MAE.index(list_of_values[0])]
  if COMPUTED_OPTION == 0:
    for i in indices:
      print(list_of_values[i])
      plot_predictions_vs_real(predictions[i],y_test[i])
  else:
    for i in indices:
        print(list_of_values[i])
        END = 24
        STARTED_MINUTE = 0
        previous = np.ones(shape=(24))
        for j in range(0,24):
            previous[j] = np.sum(X_test[i,:][60*j:60*(j+1)])
        predictions_to_plot = np.ones(shape=(END + HORIZON))
        predictions_to_plot[0:END] = previous[:]
        predictions_to_plot[END:] = predictions[i,:]
        y_test_to_plot = np.ones(shape=(END + HORIZON))
        y_test_to_plot[0:END] = previous[:]
        y_test_to_plot[END:] = y_test[i,:]
        plot_predictions_vs_real(predictions_to_plot,y_test_to_plot)

  if SPLIT_INTO_TWO_DAYS:
      index = 0
      period = X_test[::120,:]
      period_results = make_preds(model_LSTM,period)
      period_results_to_plot = np.array(period_results).reshape(HORIZON*23)
      y_test_to_plot = y_test[::120,:].reshape(HORIZON*23)
      plot_predictions_vs_real(predictions=period_results_to_plot,reals=y_test_to_plot)
      print('\033[1m' + "Predicted: " + str(np.sum(period_results_to_plot)) + '\033[0m')
      print('\033[1m' + "Reals: " + str(np.sum(y_test_to_plot)) + '\033[0m')


Resultados poblacionales cada dos horas
MSE: 63080.01
MAE: 188.94365
Ejemplos aleatorios
2.8763764


4.377115


0.52038205


1/1 - 0s - 16ms/epoch - 16ms/step


Predicted: 106007.9
Reals: 105584.72


<Figure size 640x480 with 0 Axes>

In [44]:
if MULTI_STEP_FORECAST:
    day_before = np.array(X_test[0:1,:],dtype=np.float32)
    predicted_values_day_one = np.ones(shape=(WINDOW_SIZE,1),dtype=np.float32)
    for i in range(0,WINDOW_SIZE):
        prediction = float(make_preds(model_LSTM, day_before))
        day_before[:,0:-1] = day_before[:,1:]
        day_before[:,-1] = prediction
        predicted_values_day_one[i,0] = prediction

    SECOND_DAY = X_test.shape[1]
    day_before = np.array(X_test[SECOND_DAY:SECOND_DAY+1,:],dtype=np.float32)
    predicted_values_day_two = np.ones(shape=(WINDOW_SIZE,1),dtype=np.float32)
    for i in range(0,WINDOW_SIZE):
        prediction = float(make_preds(model_LSTM, day_before))
        day_before[:,0:-1] = day_before[:,1:]
        day_before[:,-1] = prediction
        predicted_values_day_two[i,0] = prediction

In [45]:
if MULTI_STEP_FORECAST:
    print("Predecidos : " + str(int(np.sum(predicted_values_day_one[:,0]))))
    print("Reales : " + str(int(np.sum(y_test[0:WINDOW_SIZE,0]))))
    if COMPUTED_OPTION < 2:
        plot_predictions_vs_real(predicted_values_day_one[:,0],y_test[0:WINDOW_SIZE,0])

In [46]:
if MULTI_STEP_FORECAST:
    print("Predecidos : " + str(int(np.sum(predicted_values_day_two[:,0]))))
    print("Reales : " + str(int(np.sum(y_test[SECOND_DAY:,0]))))
    if COMPUTED_OPTION < 2:
        plot_predictions_vs_real(predicted_values_day_two[:,0],y_test[SECOND_DAY:,0])

In [47]:
SAVE_RESULTS = True
if SAVE_RESULTS:
  if READ_LOCAL_DATA:
    file_path = 'Resources/Resultados/Agregado/'
  else:
    file_path = "/content/drive/MyDrive/TFG/Resources/Resultados/Agregado/"
    if LOW_DATA:
        file_path += "LowData/"

    file_path += "Split"+str(SPLIT)
    documents = ['minuteY','hourY','dayY']
    file = file_path+documents[COMPUTED_OPTION]+"-predictions"+".pkl.gz"
    pickle.dump(predictions, gzip.open(file, 'wb'))
    file = file_path+documents[COMPUTED_OPTION]+"-test"+".pkl.gz"
    pickle.dump(y_test, gzip.open(file, 'wb'))
    file = file_path+documents[COMPUTED_OPTION]+"-X"+".pkl.gz"
    pickle.dump(X_test, gzip.open(file, 'wb'))